In [1]:
import programs.new_builder as nb
from pmagpy import ipmag
import os
import json
import numpy as np
import sys

import pandas as pd
from pandas import DataFrame
from pmagpy import builder
from pmagpy import validate_upload
from pmagpy import pmag

reload(nb)

pmagpy_dir = os.path.join(os.getcwd(), 'pmagpy')
if pmagpy_dir not in sys.path:
    sys.path.append(pmagpy_dir)
    
WD = os.path.join(os.getcwd(), "3_0", "McMurdo")


In [2]:
# get specimen level data from specimen table
contribution = nb.Contribution(WD, read_tables=['measurements', 'specimens', 'samples','sites'])
if 'specimens' in contribution.tables:
    spec_container = contribution.tables['specimens']
    spec_data = spec_container.df
spec_data.columns
spec_data.loc[['mc121d1', 'mc01a']][['aniso_s']]

,aniso_s
specimen,
mc121d1,0.29925 : 0.346846 : 0.353904 : -0.001806 : -0...
mc121d1,None
mc121d1,None
mc01a,None
mc01a,None


In [3]:
# find the record of a particular specimen.  if it already has aniso data, then modify in place
# otherwise add a new record and modify that....    
specimen = 'mc121d1' # a specimen with aniso data
new_s = '.3:.3:.3:0:0:0'

# add numeric index column temporarily
spec_data['num'] = range(len(spec_data))

# iterate through specimen names and update or add anisotropy data
for spec in [specimen, "mc01a"]:
    # match specimen name
    cond1 = spec_data['specimen'].str.contains(spec) == True
    # anisotropy data is present
    cond2 = spec_data['aniso_s'].notnull() == True
    cond = (cond1 & cond2)
    if len(spec_data[cond]) > 0:
        print 'update existing record', spec
        # get list of all rows for which both conditions are true
        inds = spec_data[cond]['num']
        # in case there are multiple anisotropy rows for a specimen, update each row
        for ind in inds:
            # get pre-existing data from that row
            existing_data = dict(spec_data.iloc[ind])
            # add to existing_data with values from new_data
            new_data = {'aniso_s': new_s, 'specimen': spec}
            existing_data.update(new_data)
            # update row
            spec_container.update_row(ind, existing_data)
    else:
        print 'no record found - creating new one', spec
        # add new row
        spec_container.add_row(spec, {'aniso_s': new_s, 'specimen': spec})


# sort so that all rows for a specimen are together
spec_data.sort_index(inplace=True)
# redo temporary index
spec_data['num'] = range(len(spec_data))

# see all records for mc01a & mc121d1 
# now includes 1 new anisotropy record for mc01a, and 1 updated record for mc121d1
spec_data.loc[['mc01a', specimen]][['specimen', 'aniso_s', 'num']]

update existing record mc121d1
no record found - creating new one mc01a


,specimen,aniso_s,num
specimen,,,
mc01a,mc01a,None,0
mc01a,mc01a,.3:.3:.3:0:0:0,1
mc01a,mc01a,None,2
mc121d1,mc121d1,None,283
mc121d1,mc121d1,.3:.3:.3:0:0:0,284
mc121d1,mc121d1,None,285


In [4]:
## original way:
#spec_data['aniso_s']=np.where(spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull(), new_s,spec_data['aniso_s'])
#spec_data['specimen'].str.contains(specimen) & spec_data['aniso_s'].notnull()